In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


## Recommendation Systems

According to the wikipedia definition recommendation system is a subclass of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item.





## Content Based Recommendation System.



----

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2015/08/3.jpg">

----

In this notebook I have used the Flipkart sales dataset. The main goal of the notebook is to design the recommendation system based on the description of the products.


In [ ]:
products = pd.read_csv('/kaggle/input/flipkart-products/flipkart_com-ecommerce_sample.csv')

In [ ]:
products.head()

Looking at the first five rows of the data set we can find our data set is too lengthy.

In [ ]:
len(products['product_name'])

Total number of products that we have in our dataset is exactly 2000. So it is not easy for use to know every individual products looking directly to them.

In [ ]:
len(products['product_name'].unique()),len(products['uniq_id'].unique())

Products are repeated in our dataset. By using the above line of code we can see that our data set have  12676 unique products and 20000 unique id's.

In [ ]:
products.shape

Here we have 2000 rows and 15 columns. The next step is to look at the state of our data column.

In [ ]:
products.info()

Here we can clearly see that most of the values are in the object form.

In [ ]:
products.isna().sum()

Checking at our dataset some of the columns are null. Since we are only dealing with columns `product_name` and `description` we no need to play with the rest of the colums. so they can be dropped from our data set.

In [ ]:
products['description'][0]

This is the description of the very first product in our dataset.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(max_features=None,
                     strip_accents='unicode',
                     analyzer='word',
                     min_df=10,
                     token_pattern=r'\w{1,}',
                     ngram_range=(1,3),#take the combination of 1-3 different kind of words
                     stop_words='english')#removes all the unnecessary characters like the,in etc.
products['description'] = products['description'].fillna('')

In [ ]:
#fitting the description column.
tfv_matrix = tfv.fit_transform(products['description'])#converting everythinng to sparse matrix.

In [ ]:
tfv_matrix

Looking at the above outcome, we can clearly see that our all values are converted into the spase matrix.

In [ ]:
tfv_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix,tfv_matrix)#how description of first product is related to first product and so on.

In [ ]:
sig[0]

In [ ]:
indices = pd.Series(products.index,index=products['product_name']).drop_duplicates()

In [ ]:
indices.head(20)

In [ ]:
def product_recommendation(title,sig=sig):
    indx = indices[title]
    
    #getting pairwise similarity scores
    sig_scores = list(enumerate(sig[indx]))
    
    #sorting products
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    
    #10 most similar products score
    sig_scores = sig_scores[1:11]
    
    #product indexes
    product_indices = [i[0] for i in sig_scores]
    
    #Top 10 most similar products
    return products['product_name'].iloc[product_indices]

In [ ]:
n=input("Enter the name of the product: ")
print("\nTop Recommended products are: \n")
print(product_recommendation(n).unique())


This is just a simple recommendation system developed using the flipkart dataset.